# 3.1 Parabolic model problem
In this unit we want to turn to instationary problems. And we will start with a very basic setup: implicit Euler time stepping for the convection diffusion equation. After the main part of this tutorial we have supplementary material to extend the case in several regards.



We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$

with a suitable advective field $b$ (the wind).

In [ ]:
#imports
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [ ]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
shape = Rectangle(2,2).Face().Move((-1,-1,0))
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
maxh=0.1
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=maxh))
print(mesh.GetBoundaries())
Draw(mesh)

In [ ]:
fes = H1(mesh, order=2, dirichlet="bottom|right|left|top")
u,v = fes.TnT()
time = 0.0
dt = 0.001

We define the field $b$ (the wind) as 

$$b(x,y) := (2y(1-x^2),-2x(1-y^2)).$$

In [ ]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind", vectors={"grid_size": 32}, order=3)

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix separately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [ ]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

## Implicit Euler method
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n + \Delta tf^{n+1}
$$

or in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = - \Delta t A u^n + \Delta tf^{n+1}.
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of same format as m.mat and compare number of non-zero entries and sparsity pattern:

In [ ]:
mstar = m.mat.CreateMatrix()
print(f"m.mat.nze = {m.mat.nze}, a.mat.nze={a.mat.nze}, mstar.nze={mstar.nze}")
from helper import ShowPattern
print("sparsity pattern a.mat:")
ShowPattern(a.mat,binarize=True)

In [ ]:
print("sparsity pattern mstar:")
ShowPattern(mstar,binarize=True)


To access the entries we use the vector of nonzero-entries:

In [ ]:
print(f"mstar.nze={mstar.nze}, len(mstar.AsVector())={len(mstar.AsVector())}")

In [ ]:
print(mstar.AsVector())

Using the vector we can build the linear combination of the a and the m matrix:

In [ ]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
# corresponds to M* = M + dt * A
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [ ]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f", deformation=True)
f += gaussp*v*dx
f.Assemble()

and the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x) # note that boundary conditions remain
scene = Draw(gfu,mesh,"u", deformation=True)

we define a simple time loop including some visualization sampling:

In [ ]:
def TimeStepping(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                 nsamples = 10):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    print(f"sample_int = {sample_int}")
    gfut = GridFunction(gfu.space,multidim=0)
    gfut.AddMultiDimComponent(gfu.vec)
    while time < tend - 0.5 * dt:
        res = dt * f.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        scene.Redraw()
        if cnt % sample_int == 0:
            gfut.AddMultiDimComponent(gfu.vec)
        cnt += 1; time = cnt * dt
    return gfut

In [ ]:
%%time
gfut = TimeStepping(invmstar, (1-y*y)*x)

In [ ]:
Draw(gfut, mesh, interpolate_multidim=True, animate=True, deformation=True)

## Supplementary material: VTK output

Netgen/NGSolve comes with several convenient ways to visualize your PDE solutions: the webgui, the legacy netgen GUI, ... For some use cases visualization toolboxes like [ParaView](https://www.paraview.org/) may still be able to obtain more insight or more complex visualizations from you computational results.

To put your results in a corresponding format you can use the `VTKOutput` of NGSolve. 

A `VTKOutput` object obtains a list of coefficient functions (scalar or vector) and a list of labels. Together with the mesh information these information will be put into a vtk-formatted file on every `Do`-call of the object. As VTK essentially only draws piecewise linears you may want to describe a `subdivision` argument larger `0` to obtain higher resolution of a higher order FE function.

Let's explore how we can modify the  ```TimeStepping``` function to include vtk output.


In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
time = 0.0
tend  = 2
cnt = 0

vtk = VTKOutput(mesh, coefs=[gfu], names=["sol"],filename="vtk_example1",subdivision=2)

vtk.Do(time)

while time < tend - 0.5 * dt:
    cnt += 1; time = cnt * dt
    res = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    print("\r",time,end="")
    vtk.Do(time)

In [ ]:
def TimeSteppingVTK(invmstar, initial_cond = None, t0 = 0, tend = 2, 
                 nsamples = 100, filename="results"):
    if initial_cond:
        gfu.Set(initial_cond)
    cnt = 0; time = t0
    sample_int = int(floor(tend / dt / nsamples)+1)
    
    vtk = VTKOutput(mesh, coefs=[gfu], names=["uh"],filename=filename,subdivision=2)
    vtk.Do(time)
    
    while time < tend - 0.5 * dt:
        cnt += 1; time = cnt * dt
        res = dt * f.vec - dt * a.mat * gfu.vec
        gfu.vec.data += invmstar * res
        print("\r",time,end="")
        if cnt % sample_int == 0:
            vtk.Do(time)
    # This time only return solution at final time step
    return gfu

In [ ]:
%%time
gfu = TimeSteppingVTK(invmstar, (1-y*y)*x)